In [2]:
import os 
from docx import Document
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document as docs
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.prompts import PromptTemplate
from operator import itemgetter
from langchain_core.runnables import RunnableLambda


In [3]:

def iter_block_items(parent):
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    elif isinstance(parent, _Row):
        parent_elm = parent._tr
    else:
        raise ValueError("something's not right")
    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

def get_tabla_de_bandejas(doc):
    string = "Generación de Estructuras de Bandejas"
    seccion_hallada = False
    tabla_hallada = False
    for block in iter_block_items(doc):
        if tabla_hallada:
            break

        if isinstance(block, Paragraph) and not seccion_hallada:
            if string.strip() in block.text:
                seccion_hallada = True
        if seccion_hallada:
            if isinstance(block, Table):
                tabla_hallada = True
                tabla = block
    if tabla_hallada:
        tabla_lista = []
        for row in tabla.rows:
                tabla_lista.append([cell.text for cell in row.cells])
        return tabla_lista
    else:
        return -1
    


def get_tabla_de_campos(doc,bandeja):
    string = "Diseño de Tablas de Bandejas"
    seccion_hallada = False
    tabla_hallada = False
    bandeja_hallada = False

    for block in iter_block_items(doc):
        if tabla_hallada:
            break

        if isinstance(block, Paragraph) and not seccion_hallada:
            if string.strip() in block.text:
                seccion_hallada = True

        if isinstance(block, Paragraph) and seccion_hallada:
            if bandeja.strip() in block.text:
                bandeja_hallada = True
        if bandeja_hallada:
            if isinstance(block, Table):
                tabla_hallada = True
                tabla = block
    if tabla_hallada:
        tabla_lista = []
        for row in tabla.rows:
                tabla_lista.append([cell.text.strip() for cell in row.cells])
        return tabla_lista
    else:
        return -1

In [4]:
path = "C:/Users/iboero/Chat_w_PDF/data/raw_docs/Migracion/"
doc_paths = [x for x in os.listdir(path) if x[-5:]==".docx"]

lista_tabla_bandejas = {}
lista_elementos_bandejas = {}

for doc_path in doc_paths:
    doc = Document(path + doc_path)
    tabla_bandejas = get_tabla_de_bandejas(doc)
    lista_tabla_bandejas[doc_path[:-5]] = tabla_bandejas
    # print(tabla_bandejas)
    bandejas =  tabla_bandejas[1:]
    bandejas = [sublista[0] for sublista in bandejas]

    elementos_bandejas = {}
    for bandeja in bandejas:
        tabla = get_tabla_de_campos(doc, "Tabla "+bandeja.strip())
        if tabla != -1:
            elementos_bandejas[bandeja] = tabla
    lista_elementos_bandejas[doc_path[:-5]] = elementos_bandejas
    # print(doc_path, elementos_bandejas.keys())

In [5]:
class bandeja:
    def __init__(self,codigo,descripcion):
        self.codigo = codigo
        self.descripcion = descripcion
        self.largo = 0
        self.campos = []

    def add_campo(self,campo):
        self.largo += 1
        self.campos.append(campo)

    def get_campo(self,campo):
        for c in self.campos:
            if c.codigo.strip() == campo.strip():
                return c
        return -1
    

    
class campo:
    def __init__(self,codigo,descripcion,tipo,idx):
        self.codigo = codigo
        self.descripcion = descripcion
        self.idx = idx
        self.tipo = tipo

    
class manual:
    def __init__(self,codigo, desc_bandejas,lista_campos):
        self.codigo = codigo
        self.bandejas = self.generate_bandejas(desc_bandejas,lista_campos)

    def generate_bandejas(self,desc_bandejas,lista_campos):
        bjs_objects = []
        for b in desc_bandejas[1:]:
            if b[0] in lista_campos.keys():
                bj = bandeja(codigo=b[0],descripcion=b[1])
                for c in lista_campos[b[0]][1:]:
                    if len(c)>=4:
                        desc_extra = c[3]
                    else:
                        desc_extra = ''
                    c_obj = campo(codigo=c[0], descripcion=c[1]+' '+desc_extra,tipo=c[2],idx=bj.largo)
                    bj.add_campo(c_obj)
                bjs_objects.append(bj)
        return bjs_objects
    
    def get_bandeja(self,bandeja):
        for b in self.bandejas:
            if b.codigo.strip() == bandeja.strip():
                return b
        return -1


In [6]:
manuales_obj = {}
for m in lista_elementos_bandejas.keys():
    m_obj = manual(m,lista_tabla_bandejas[m],lista_elementos_bandejas[m])
    manuales_obj[m] = m_obj



def get_desc_bandejas(manual):
    manual = manuales_obj[manual]
    string = ""
    for b in manual.bandejas:
        string +=  b.codigo + ': ' + b.descripcion + '\n'
    return string

In [7]:
import dill as pkl
filename = 'manuales_object.pkl'
with open(filename, 'wb') as file:
    pkl.dump(manuales_obj, file)

In [8]:
db_ret = Chroma(persist_directory="db_campos", embedding_function=OpenAIEmbeddings())
db_ret.delete_collection()

d = []

for m_name in manuales_obj:
    m = manuales_obj[m_name]
    for b in m.bandejas:
        for c in b.campos:
            print({"manual":m.codigo,"bandeja":b.codigo, "campo":c.codigo},c.descripcion)
            d.append(docs(page_content=c.descripcion, metadata={"manual":m.codigo,"bandeja":b.codigo, "campo":c.codigo,"tipo":c.tipo}))
            
embedding_function = OpenAIEmbeddings()
db = Chroma.from_documents(d, embedding_function, persist_directory="db_campos")

{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', 'campo': 'BnjEmp'} BnjEmp Empresa
{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', 'campo': 'BnjCod'} Código de Bandeja Código de bandeja.
{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', 'campo': 'BnjSuc'} Sucursal Sucursal donde se otorgó el Acuerdo
{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', 'campo': 'BnjMda'} Moneda Moneda en el que se otorgó.
{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', 'campo': 'BnjPap'} Papel No corresponde
{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', 'campo': 'BnjCta'} Cuenta Cuenta cliente.
{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', 'campo': 'BnjOpe'} Operación Número identificador del Contrato de Acuerdo.
{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', 'campo': 'BnjSbOp'} Suboperación Suboperación = Subcuenta
{'manual': 'Manual Acuerdos de Sobregiro', 'bandeja': 'BNJ002', '

In [9]:
db_ret = Chroma(persist_directory="db_campos", embedding_function=OpenAIEmbeddings())
retriever = db_ret.as_retriever()

In [10]:
def parse_campos(pregunta):
    campos =  db_ret.similarity_search(pregunta,filter={"manual":manual},k=10)
    print(campos)
    string = ""
    for c in campos:
        md = c.metadata
        string += f'- Campo: {md["campo"]}. Descripción: {c.page_content}. Pertenece a la bandeja: {md["bandeja"]}. Es de tipo: {md["tipo"]} \n\n'
    return string



prompt = PromptTemplate(template=""" 
Sos un asistente de chat para el proceso de migración de Bantotal.
Se presenta una breve introducción sobre el proceso de migración:

<introduccion>
En el Sistema existen dos tipos de programas claramente diferenciados:
Programas de Control: son aquellos  que verifican que los datos que se pretenden grabar en las tablas sean aptos. Indican cada registro con un código de estado (si es válido o no).
Programas de Vuelco: son aquellos que toman la información controlada por los programas de Control, y la graban en las tablas.

Los programas de Control y de Vuelco actúan sobre determinados registros, que se identifican como pertenecientes a una Bandeja. Es decir, una Bandeja identifica a un conjunto de registros, que pueden pertenecer a una o más tablas y ser “vistas” de estas.

Las Bandejas se crean específicamente para cada cliente de acuerdo a sus necesidades. Se entiende que una Bandeja reúne un conjunto de operaciones de un mismo tipo de producto (por ejemplo: Cajas de Ahorro).

Unificando estos dos conceptos descritos, este Sistema contiene programas que procesan determinada información, que se denomina Bandeja.
</introduccion>


Se cuenta con las siguientes bandejas:
<bandejas>
{desc_bandejas}
</bandejas>                

Se tiene una lista de potenciales registros para guardar la información:
                        
<resgistros>
{campos}
</resgistros>

La información a guardar es:
información: {pregunta}
                        
Elegir cual o cuales de los resgistro o resgistros es adecuado para guardar esa información.
Luego explicar que información guarda la bandeja a la que pertenece el resgistro, y desarrollar sobre que significa el tipo del resgistro.
En caso que haya varios resgistros posibles, explicar la razon por la cual seleccionaste cada uno y decir cual es el más probable entre los elegidos.
""", input_variables=["pregunta","campos","desc_bandejas"])

In [197]:
manual = 'Manual Tarjetas de Débito'
desc_bandejas = get_desc_bandejas(manual)
pregunta = "Fecha de vencimiento de una tarjeta"
retriever = db_ret.as_retriever(filter={"manual":manual},k=10)

llm = ChatOpenAI(temperature=0.0)
chain = {"campos":itemgetter("pregunta") | RunnableLambda(parse_campos),"desc_bandejas":itemgetter("manual") | RunnableLambda(get_desc_bandejas), "pregunta":itemgetter("pregunta")} | prompt | llm | StrOutputParser()

In [198]:
print(chain.invoke({"pregunta":pregunta,"manual":manual}))

[Document(page_content='Identificador de Tarjeta ', metadata={'bandeja': 'BNJ040', 'campo': 'BNJ040IDT', 'manual': 'Manual Tarjetas de Débito', 'tipo': 'N ( 10.0 )'}), Document(page_content='Fecha de autorizacion ', metadata={'bandeja': 'BNJ041', 'campo': '', 'manual': 'Manual Tarjetas de Débito', 'tipo': 'D'}), Document(page_content='Vencimiento Bonificacion ', metadata={'bandeja': 'BNJ040', 'campo': 'BNJ040BVT', 'manual': 'Manual Tarjetas de Débito', 'tipo': 'D'}), Document(page_content='Fecha de Ultimo Uso Fecha de Ultimo Uso', metadata={'bandeja': 'BNJ040', 'campo': 'BNJ040FUU', 'manual': 'Manual Tarjetas de Débito', 'tipo': 'D'}), Document(page_content='Numero Tarjeta Numero Tarjeta', metadata={'bandeja': 'BNJ041', 'campo': 'BNJ041Nro', 'manual': 'Manual Tarjetas de Débito', 'tipo': 'C (19)'}), Document(page_content='Numero Tarjeta Numero Tarjeta', metadata={'bandeja': 'BNJ040', 'campo': 'BNJ040NRO', 'manual': 'Manual Tarjetas de Débito', 'tipo': 'C (19)'}), Document(page_content=